In [2]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import sklearn, sys
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston

## Versions

In [3]:
libraries = (('Matplotlib', matplotlib), ('Numpy', np),
             ('Pandas', pd), ('Sklearn', sklearn))

print("Python version:", sys.version, '\n')
for lib in libraries:
    print('{0} version: {1}'.format(lib[0], lib[1].__version__))

Python version: 3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)] 

Matplotlib version: 2.2.2
Numpy version: 1.14.3
Pandas version: 0.22.0
Sklearn version: 0.19.1


## Introduction

Tutorial to understand the trade-offs between overfitting and underfitting
Our first step is to read in the data and prep it for modeling. 

## Get & Prep Data

In [4]:
# get data using a sample boston dataset from machine learning library
boston = load_boston()
data = boston.data
target = boston.target

# train/test split
X_train, X_test, y_train, y_test = train_test_split(data, 
                                                    target, 
                                                    shuffle=True,
                                                    test_size=0.2, 
                                                    random_state=15)

## Functions to Calculate Train Error & Test Error

In [5]:
def calc_train_error(X_train, y_train, model):
    '''returns in-sample error for already fit model.'''
    predictions = model.predict(X_train)
    mse = mean_squared_error(y_train, predictions)
    rmse = np.sqrt(mse)
    return mse

In [7]:
def calc_validation_error(X_test, y_test, model):
    '''returns out-of-sample error for already fit model.'''
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    return mse

In [8]:
def calc_metrics(X_train, y_train, X_test, y_test, model):
    '''fits model and returns the RMSE for in-sample error and out-of-sample error'''
    model.fit(X_train, y_train)
    train_error = calc_train_error(X_train, y_train, model)
    validation_error = calc_validation_error(X_test, y_test, model)
    return train_error, validation_error

## Practical

I'm building a linear regression model of the Forest Fire dataset to investigate whether my model is underfitting, overfitting, or fitting just right. If it's under or overfitting, we'll look at one way we can correct that.

Time to build the model.

In [9]:
lr = LinearRegression(fit_intercept=True)

train_error, test_error = calc_metrics(X_train, y_train, X_test, y_test, lr)
train_error, test_error = round(train_error, 3), round(test_error, 3)

print('train error: {} | test error: {}'.format(train_error, test_error))
print('train/test: {}'.format(round(test_error/train_error, 1)))

train error: 21.874 | test error: 23.817
train/test: 1.1


Hmm, my training error is considerably lower than the test error. In fact, the test error is 1.5 times worse. 

Which region does that put us in? That's right, it's in the high variance region, which means my model is overfitting. Again, that means my model has too much complexity. You're probably thinking, hey wait, no we're not. Drop a feature or two and then recalculate train error and test error. My response is simply: NO, DON'T EVER DO THAT. NEVER. FOR ANY REASON. 

Why not?

Because your test set is no longer a test set. You are using it to train your model. It's the same as if you trained your model on the all the data from the outset. Seriously, don't do this. Unfortunately, practicing data scientists do this sometimes; it's one of the worst things you can do. You're almost guaranteed to produce a model that cannot generalize. 

So what do we do?

We need to go back to the beginning. We need to split our data into three datasets: training, validation, test. Remember, the test set is data you don't touch until you're happy with your model. The test set is used ONE time to see how your model will generalize. That's it.

Okay, let's take a look at this thing called a validation set.

## Validation Set

Three datasets from one seems like a lot of work but I promise it's worth it. 

First, let's see how to do this in practice.

In [8]:
# intermediate/test split (gives us test set)
X_intermediate, X_test, y_intermediate, y_test = train_test_split(data, 
                                                                  target, 
                                                                  shuffle=True,
                                                                  test_size=0.2, 
                                                                  random_state=15)

# train/validation split (gives us train and validation sets)
X_train, X_validation, y_train, y_validation = train_test_split(X_intermediate,
                                                                y_intermediate,
                                                                shuffle=False,
                                                                test_size=0.25,
                                                                random_state=2018)

# delete intermediate variables
del X_intermediate, y_intermediate

# print proportions
print('train: {}% | validation: {}% | test {}%'.format(round(len(y_train)/len(target),2),
                                                       round(len(y_validation)/len(target),2),
                                                       round(len(y_test)/len(target),2)))

train: 0.6% | validation: 0.2% | test 0.2%


Time to fit and tune our model. 

## Model Tuning

I want to decrease complexity. One way to do this is by using *regularization*. I won't go into the nitty gritty now because that will be for a future post. Just know that regularization is constrained optimization that imposes limits on determining model parameters. It effectively allows me to add bias to a model that's overfitting. I can control the amount of bias with a hyperparameter called *lambda* or *alpha* - you'll see both, though sklearn uses alpha because lambda is a keyword - that defines regularization strength.

In [9]:
alphas = [0.001, 0.01, 0.1, 1, 10]
print('All errors are RMSE')
print('-'*76)
for alpha in alphas:
    # instantiate and fit model
    ridge = Ridge(alpha=alpha, fit_intercept=True, random_state=99)
    ridge.fit(X_train, y_train)
    # calculate errors
    new_train_error = mean_squared_error(y_train, ridge.predict(X_train))
    new_validation_error = mean_squared_error(y_validation, ridge.predict(X_validation))
    new_test_error = mean_squared_error(y_test, ridge.predict(X_test))
    # print errors as report
    print('alpha: {:7} | train error: {:5} | val error: {:6} | test error: {}'.
          format(alpha,
                 round(new_train_error,3),
                 round(new_validation_error,3),
                 round(new_test_error,3)))

All errors are RMSE
----------------------------------------------------------------------------
alpha:   0.001 | train error: 22.93 | val error: 19.796 | test error: 23.959
alpha:    0.01 | train error: 22.93 | val error: 19.792 | test error: 23.944
alpha:     0.1 | train error: 22.945 | val error: 19.779 | test error: 23.818
alpha:       1 | train error: 23.324 | val error: 20.135 | test error: 23.522
alpha:      10 | train error: 24.214 | val error: 20.958 | test error: 23.356


#### Setup Data, Model, & Calculate Errors

In [10]:
# train/test split
X_train, X_test, y_train, y_test = train_test_split(data, 
                                                    target, 
                                                    shuffle=True,
                                                    test_size=0.2, 
                                                    random_state=15)

# instantiate model
ridge = Ridge(alpha=0.11, fit_intercept=True, random_state=99)

# fit and calculate errors
new_train_error, new_test_error = calc_metrics(X_train, y_train, X_test, y_test, ridge)
new_train_error, new_test_error = round(new_train_error, 3), round(new_test_error, 3)

#### Report Errors

In [11]:
print('ORIGINAL ERROR')
print('-' * 40)
print('train error: {} | test error: {}\n'.format(train_error, test_error))
print('ERROR w/REGULARIZATION')
print('-' * 40)
print('train error: {} | test error: {}'.format(new_train_error, new_test_error))

ORIGINAL ERROR
----------------------------------------
train error: 21.874 | test error: 23.817

ERROR w/REGULARIZATION
----------------------------------------
train error: 21.883 | test error: 23.673


A very small increase in training error coupled with a small but larger in magnitude decrease in test error. We're definitely moving in the right direction. Perhaps not quite the magnitude of change we expected, but we're simply trying to prove a point here. Remember this is a tiny dataset. Also remember I said we can do better by using something called *Cross-Validation*. Now's the time to talk about that.

## Cross-Validation

## Sklearn & CV

There's two ways to do this in sklearn, pending what you want to get out of it. 

The first method I'll show you is `cross_val_score`, which works beautifully if all you care about is validation error.

The second method is `KFold`, which is perfect if you want train and validation errors.

Let's try a new model called *LASSO* just to keep things interesting. 

### cross_val_score

In [12]:
from sklearn.model_selection import cross_val_score

alphas = [1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1]

val_errors = []
for alpha in alphas:
    lasso = Lasso(alpha=alpha, fit_intercept=True, random_state=77)
    errors = np.sum(-cross_val_score(lasso, 
                                     data, 
                                     y=target, 
                                     scoring='neg_mean_squared_error', 
                                     cv=10, 
                                     n_jobs=-1))
    val_errors.append(np.sqrt(errors))

Let's checkout the validation errors associated with each alpha.

In [13]:
# RMSE
print(val_errors)

[18.64401379981868, 18.636528438323769, 18.578057471596566, 18.503285318281634, 18.565586130742307, 21.412874355105991]


Which value of alpha gave us the smallest validation error?

In [14]:
print('best alpha: {}'.format(alphas[np.argmin(val_errors)]))

best alpha: 0.1


### KFold

In [15]:
from sklearn.model_selection import KFold

K = 10
kf = KFold(n_splits=K, shuffle=True, random_state=42)

for alpha in alphas:
    train_errors = []
    validation_errors = []
    for train_index, val_index in kf.split(data, target):
        
        # split data
        X_train, X_val = data[train_index], data[val_index]
        y_train, y_val = target[train_index], target[val_index]

        # instantiate model
        lasso = Lasso(alpha=alpha, fit_intercept=True, random_state=77)
        
        #calculate errors
        train_error, val_error = calc_metrics(X_train, y_train, X_val, y_val, lasso)
        
        # append to appropriate list
        train_errors.append(train_error)
        validation_errors.append(val_error)
    
    # generate report
    print('alpha: {:6} | mean(train_error): {:7} | mean(val_error): {}'.
          format(alpha,
                 round(np.mean(train_errors),4),
                 round(np.mean(validation_errors),4)))


alpha: 0.0001 | mean(train_error): 21.8217 | mean(val_error): 23.3633
alpha:  0.001 | mean(train_error): 21.8221 | mean(val_error): 23.3647
alpha:   0.01 | mean(train_error): 21.8583 | mean(val_error): 23.4126
alpha:    0.1 | mean(train_error): 22.9727 | mean(val_error): 24.6014
alpha:      1 | mean(train_error): 26.7371 | mean(val_error): 28.236
alpha:   10.0 | mean(train_error):  40.183 | mean(val_error): 40.9859


Comparing the output of *cross_val_score* to that of *KFold*, we can see that the general trend holds - an alpha of 10 results in the largest validation error. You may wonder why we get different values. The reason is that the data was split differently. We can control the splitting procedure with KFold but not cross_val_score.

## Summary

I discussed the Bias-Variance Tradeoff where a high bias model is one that is underfit while a high variance model is one that is overfit. I splitted data into three groups for tuning purposes. Specifically, the three groups are train, validation, test. Remember the test set is used only one time to check how well a model generalizes on data it's never seen. This three-group split works exceedingly well for large datasets, not for small to medium-sized datasets, though. In that case, use cross-validation. CV can help you tune your models and extract as much signal as possible from the small data sample. Remember, with CV you don't need a test set. By using a K-fold approach, you get the equivalent of K-test sets by which to check validation error. I think helps to diagnose where you're at in the Bias-Variance regime.